In [1]:
import pickle
import click


CONFIG = {'min_3dash_threshold': 10, 'min_3dash_peak_separation': 30, 'min_3dash_end_separation': 100, 
          'min_5dash_threshold': 10, 'min_5dash_peak_separation': 30, 'min_5dash_start_separation': 100, 
          'max_5dash_offset': 200}

In [14]:
def clusterMean(cluster):
    totalSites=0
    totalPosition=0
    #The mean is calculated proportionnaly to the number of sites or each position
    for site in cluster:
        totalSites += int(site["coverage"])
        totalPosition = totalPosition+(int(site["position"])*int(site["coverage"]))

    return round(totalPosition/totalSites)

In [3]:
in3dash_file = '../transitional/fhh.positive.3dash_positions.tsv'
in5dash_file = '../transitional/fhh.positive.3dash_positions.tsv'
output = '../transitional/fhh.positive.genelist.bed'
chromosome_data = '../transitional/fhh.positive_genelist_incomplete.bed'

In [4]:
if 'positive' in in3dash_file and 'positive' in in5dash_file:
    strand = '+'
elif 'negative' in in3dash_file and 'negative' in in5dash_file:
    strand = '-'
else:
    raise NameError('input error')

with open(f'{chromosome_data}.pydata', 'rb') as f:
    chromosomeList = pickle.load(f)

In [6]:
if 'positive' in in3dash_file and 'positive' in in5dash_file:
    strand = '+'
elif 'negative' in in3dash_file and 'negative' in in5dash_file:
    strand = '-'
else:
    raise NameError('input error')

with open(f'{chromosome_data}.pydata', 'rb') as f:
    chromosomeList = pickle.load(f)

In [16]:
for dash in ('3', '5'):
    #Read the 3' or 5' coverage file o find clusters of start sites or end sites
    currentDashSite = {"chr":None, "position":None, "coverage":None}  #The line read currently
    lastDashSite = None  #The last line that had a coverage over theset threshold
    currentCluster = []  #The current cluster of results
    currentChr = None  #The current chromosome name
    currentChrList = []  #The list 3' and 5' sites for the current chromosome
    dashSiteList = {}  #The full list of relevant 3' or 5' sites
    if dash == "3":
        sourceFile = in3dash_file
    elif dash == "5":
        sourceFile = in5dash_file
    # @sourceFile: reads 3'/5' 端覆盖度
    # @sourceFile format: chro, pos, cov
    #                     1  5743  1
    with open(sourceFile, "r") as dashList:
        for line in dashList:
            currentDashSite["chr"],  currentDashSite["position"],  currentDashSite["coverage"] = line.rstrip().split("\t")
            #First check if we are entering a new chromosome
            if currentDashSite["chr"] != currentChr:
                #if so, store the list of sites for the previous chromosome, then empty the list and reset the variables
                if currentChr is not None:
                    if currentCluster != []:
                        currentChrList.append(clusterMean(currentCluster))
                    dashSiteList[currentChr] = list(currentChrList)
                currentCluster = []
                currentChr=currentDashSite["chr"]
                currentChrList = []
                lastDashSite = None

            #lines with a coverage less than a threshold (min_3dash_threshold) are ignored
            if int(currentDashSite["coverage"]) > CONFIG[f"min_{dash}dash_threshold"]:
                #Peaks within a distance of each other (min_3dash_peak_separation) are clustered together
                if lastDashSite is None or int(currentDashSite["position"])-int(lastDashSite["position"]) < CONFIG[f"min_{dash}dash_peak_separation"]:
                    currentCluster.append(dict(currentDashSite))
                #If the peak is far enough from the last cluster, register the mean position for the peak and reset the cluster with our new result
                else:
                    #The mean position is stored in the current chromosome list
                    if currentCluster != []:
                        currentChrList.append(clusterMean(currentCluster))
                    # 重置，计算下一个cluster
                    currentCluster = []
                    currentCluster.append(dict(currentDashSite))
                #Once the site has been saved, keep it to compare it to the next
                lastDashSite = dict(currentDashSite)

    #Add the last chromosome list
    if currentChr is not None:
        currentChrList.append(clusterMean(currentCluster))
        dashSiteList[currentChr] = list(currentChrList)
    #The dashSiteList now contains every cut site given by our 3' or 5' file, organized by chomosome name
    # dashSiteList 为上面结果汇总，为每一挑染色体上，每一个cluster的均值

    #Copy the dash site list to the correct variable
    if dash == "3":
        dashSiteList3=dict(dashSiteList)
    elif dash == "5":
        dashSiteList5=dict(dashSiteList)


In [8]:
chromosomeList['1'][:10]

[{'start': 3859, 'end': 5903, 'strand': '+'},
 {'start': 26650, 'end': 31229, 'strand': '+'},
 {'start': 52216, 'end': 54730, 'strand': '+'},
 {'start': 72327, 'end': 74108, 'strand': '+'},
 {'start': 75723, 'end': 76678, 'strand': '+'},
 {'start': 88898, 'end': 90618, 'strand': '+'},
 {'start': 91421, 'end': 95667, 'strand': '+'},
 {'start': 96028, 'end': 97424, 'strand': '+'},
 {'start': 97475, 'end': 99241, 'strand': '+'},
 {'start': 99920, 'end': 101841, 'strand': '+'}]

In [18]:
dashSiteList['1'][:10]

[74049, 74095, 89752, 95665, 99239, 116153, 118543, 118644, 118744, 138083]